In [4]:
import os
import re
import csv
import sys
import csv
import random
import ast

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [20]:
# load data
recs = []
with open("../all-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
print len(recs)
recs[0]

39191


{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'year': '2008'}

## 1. Clean

In [3]:
# check for broken csv rows
for i in recs:
    if not i['year'].startswith('2'):
        print i['text']

In [33]:
# ordering
"""
numbs = []
for i in recs[:500]:
    n = i['text'].split()[0]
    n = n.split('.')
    i['n1'] = n[0]
    i['n2'] = n[1]
    if len(n) == 3:
        i['n3'] = n[2]
    else:
        i['n3'] = 0
"""

## 2. Sample

In [28]:
f = open('../sample-ids.txt','r')
sampled_ids = f.readlines()
f.close()
sampled_ids = [ast.literal_eval(i) for i in sampled_ids]
sampled_ids = [item for sublist in sampled_ids for item in sublist]

In [29]:
# remove already sampled recs
for i in recs:
    if i['id'] in sampled_ids:
        recs.remove(i)
len(recs)

38991

In [32]:
# sample
rec_ids = [int(i['id']) for i in recs] # get ids of remaining recs
ids = random.sample(rec_ids, 100) # sample those ids

sample = []
for i in recs:
    if int(i['id']) in ids:
        sample.append(i)
len(sample)

100

In [33]:
# delete data we don't want to include
for i in sample:
    del i['institutions']
    del i['theme']
    del i['action']
    del i['notes']
    
sample[0]

{'From': 'Sudan',
 'decision': 'support',
 'id': '1',
 'text': '69.2 That Algeria share its experience in the fight against terrorism and pursue its efforts to combat poverty and promote economic, social and cultural rights with a view to achieving the Millennium Development Goals by 2015 (Sudan). );',
 'to': 'Algeria',
 'year': '2008'}

In [34]:
#writing column headings
keys = sample[0].keys()

#writing the rest
with open('sample-matt-sentiment.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sample)

In [35]:
# add ids to the list of sampled data
ids = [str(i) for i in ids]
with open('../sample-ids.txt', 'a') as output_file:
    output_file.write("\n" + str(ids))

## 3. Test

In [16]:
# load data
recs = []
with open("../all-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
len(recs)
recs[0]

{'From': 'Kuwait',
 'action': "['establish']",
 'decision': 'support',
 'id': '0',
 'institutions': '[]',
 'notes': '',
 'text': '69.1 The establishment of an international round table to discuss the interrelation between security and fundamental freedoms (Kuwait). );',
 'theme': '[]',
 'to': 'Algeria',
 'year': '2008'}

In [17]:
# load Erin
erin = []
with open("erin-data.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                erin.append(row)
            
erin[1]

{'From': 'Tunisia',
 'action': "['continue']",
 'decision': 'consider',
 'id': '690',
 'institutions': '[]',
 'notes about coding': '',
 'sentiment': '3',
 'text': '58.14 To continue to work on improving the situation of children\x90s rights (Algeria, Tunisia);',
 'theme': "['children']",
 'to': 'Romania',
 'year': '2008'}

In [18]:
# making literal lists
for i in erin:
    i['theme'] = ast.literal_eval(i['theme'])
    i['institutions'] = ast.literal_eval(i['institutions'])
    i['action'] = ast.literal_eval(i['action'])

erin[1]

{'From': 'Tunisia',
 'action': ['continue'],
 'decision': 'consider',
 'id': '690',
 'institutions': [],
 'notes about coding': '',
 'sentiment': '3',
 'text': '58.14 To continue to work on improving the situation of children\x90s rights (Algeria, Tunisia);',
 'theme': ['children'],
 'to': 'Romania',
 'year': '2008'}

In [19]:
# strip whitespace:
for i in erin:
    i['theme'] = [x.strip() for x in i['theme']]
    i['institutions'] = [x.strip() for x in i['institutions']]
    i['action'] = [x.strip() for x in i['action']]

In [20]:
# get ids
ids = [rec['id'] for rec in erin]

In [22]:
# get original sample
sample = []
for i in recs:
    if i['id'] in ids:
        sample.append(i)
sample[0]  

{'From': 'Mexico',
 'action': '[]',
 'decision': 'support',
 'id': '92',
 'institutions': '[]',
 'notes': '',
 'text': '83.8 Enhance access to justice as well as improve the judicial system (Mexico);',
 'theme': "['judiciary']",
 'to': 'Brazil',
 'year': '2008'}

In [23]:
for i in sample:
    i['theme'] = ast.literal_eval(i['theme'])
    i['institutions'] = ast.literal_eval(i['institutions'])
    i['action'] = ast.literal_eval(i['action'])

sample[1]

{'From': 'Tunisia',
 'action': ['continue'],
 'decision': 'consider',
 'id': '690',
 'institutions': [],
 'notes': '',
 'text': '58.14 To continue to work on improving the situation of children\xcds rights (Algeria, Tunisia);',
 'theme': ['children'],
 'to': 'Romania',
 'year': '2008'}

In [24]:
# zip
matching = []
for i in range(100):
    dic = erin[i].copy()
    
    # Theme
    dic['theme-erin'] = erin[i]['theme']
    dic['theme-rochelle'] = sample[i]['theme']
    del dic['theme']
    
    if set(dic['theme-erin']) == set(dic['theme-rochelle']):
        dic['theme-match'] = True
    elif set(dic['theme-erin']).intersection(dic['theme-rochelle']):
        dic['theme-match'] = "Partial"
    else:
        dic['theme-match'] = False
    
    # institutions
    dic['institution-erin'] = erin[i]['institutions']
    dic['institution-rochelle'] = sample[i]['institutions']
    del dic['institutions']
    
    if set(dic['institution-erin']) == set(dic['institution-rochelle']):
        dic['institution-match'] = True
    elif set(dic['institution-erin']).intersection(dic['institution-rochelle']):
        dic['institution-match'] = "Partial"
    else:
        dic['institution-match'] = False
    
    # actions
    dic['action-erin'] = erin[i]['action']
    dic['action-rochelle'] = sample[i]['action']
    del dic['action']
    
    if set(dic['action-erin']) == set(dic['action-rochelle']):
        dic['action-match'] = True
    elif set(dic['action-erin']).intersection(dic['action-rochelle']):
        dic['action-match'] = "Partial"
    else:
        dic['action-match'] = False
    
    matching.append(dic)

In [25]:
# make lists of matches, etc
theme_match = [i for i in matching if i['theme-match'] == True]
theme_partial = [i for i in matching if i['theme-match'] == "Partial"]
theme_false = [i for i in matching if i['theme-match'] == False]

institution_match = [i for i in matching if i['institution-match'] == True]
institution_partial = [i for i in matching if i['institution-match'] == "Partial"]
institution_false = [i for i in matching if i['institution-match'] == False]

action_match = [i for i in matching if i['action-match'] == True]
action_partial = [i for i in matching if i['action-match'] == "Partial"]
action_false = [i for i in matching if i['action-match'] == False]

In [26]:
print "Theme" + "\n"
print "Match: " + str(len(theme_match))
print "Partial: " + str(len(theme_partial))
print "False: " + str(len(theme_false))
print "\n"

print "Institutions" + "\n"
print "Match: " + str(len(institution_match))
print "Partial: " + str(len(institution_partial))
print "False: " + str(len(institution_false))
print "\n"

print "Action" + "\n"
print "Match: " + str(len(action_match))
print "Partial: " + str(len(action_partial))
print "False: " + str(len(action_false))
print "\n"

Theme

Match: 76
Partial: 19
False: 5


Institutions

Match: 95
Partial: 2
False: 3


Action

Match: 84
Partial: 6
False: 10




In [181]:
for i in action_partial:
    print i['id'] + ": " + i['text']
    print "ERIN: " + str(i['action-erin'])
    print "ROCHELLE: " + str(i['action-rochelle']) + "\n"

4385: 57.8.1 and take all appropriate measures, in the fields of legislation, implementation and awareness-raising, to tackle domestic violence against women and children (Italy);
ERIN: ['take-measures']
ROCHELLE: ['implement', 'take-measures']

9370: 101.12. Continue to implement the core elements of the Children�s Act, which is a great step forwards in the achievement of the Millennium Development Goals (Angola);
ERIN: ['continue']
ROCHELLE: ['continue', 'implement']

14471: 92.46. Carry out a review of norms and practices relating to freedom of belief in order to harmonize domestic laws with international standards established under ICCPR (Mexico);
ERIN: ['harmonize']
ROCHELLE: ['harmonize', 'establish']

20347: 108.30 Take necessary measures to ensure effective implementation of laws guaranteeing free registration of births nationwide, including by educating families and communities on the importance of birth registration in order to contribute, among other things, to eliminating t

## 4. Turn to Binary SpreadSheet

In [27]:
institutions = ['core', 'CRPD', 'ICCPR', 'eu-vaw', 'geneva', 'ICC', 'ILO', 'ICESR', 'achr', 'CAT', 'CRC', 'echr', '1954-stateless', 'traffick', 'CTOC', 'ICMW', 'ICERD', 'eu-child', 'sp-pr', 'PP', 'kampala', '1951refugees', 'pol-women', 'CPED', 'uncac', 'ottawa', '1961-stateless', 'CEDAW', 'unesco-ed', 'hague', 'genocide', 'HRC', 'unesco-expr', 'int-cord', 'frame-minorities', 'ind169', 'OP', 'ic-mercenaries', 'eu-migrant', 'un-indig', 'vienna', 'ouagadougou']

In [28]:
actions = ['accede', 'ratify', 'take-measures', 'sign', 'report', 'continue', 'withdraw', 'harmonize', 'party', 'implement', 'establish']

In [29]:
theme = ['death-pen', 'infrastructure ', 'intelligence', 'prisoners', 'elderly', 'culture', 'education', 'children', 'civil-so', 'disappearances', 'police', 'hiv', 'privacy', 'discrimination', 'impunity', 'governance', 'labor', 'environment', 'religion', 'foreign-movement', 'health', 'speech', 'domest-movement', 'terrorism', 'paramilitary', 'corruption', 'sex-violence', 'civil-conflict', 'race', 'torture', 'food', 'extra-judicial', 'judiciary', 'indigenous', 'lgbt', 'self-determ', 'poverty', 'ethnic', 'women', 'land', 'reproductive', 'post-conflict', 'disabilities', 'migrants', 'pol-imprisonment', 'socialism', 'trafficking', 'military', 'property']

In [30]:
erin[0].keys()

['From',
 'sentiment',
 'text',
 'decision',
 'id',
 'to',
 'theme',
 'year',
 'action',
 'notes about coding',
 'institutions']

In [31]:
for i in sample:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0 
    
    del i['to']
    del i['From']
    del i['text']
    del i['notes']
    del i['action']
    del i['theme']
    del i['decision']
    del i['institutions']
    del i['year']

In [32]:
for i in erin:
    for s in institutions:
        if s in i['institutions']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in actions:
        if s in i['action']:
            i[s] = 1
        else:
            i[s] = 0
            
    for s in theme:
        if s in i['theme']:
            i[s] = 1
        else:
            i[s] = 0  
            
    del i['to']
    del i['From']
    del i['text']
    del i['year']
    del i['institutions']
    del i['action']
    del i['theme']
    del i['decision']
    del i['notes about coding']
    del i['sentiment']

In [33]:
## writing sample

#writing column headings
keys = sample[0].keys()

#writing the rest
with open('sample-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(sample)

In [34]:
## writing erin

#writing column headings
keys = erin[0].keys()

#writing the rest
with open('erin-binary.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(erin)